#  영화 리뷰 감성분석

In [1]:
from google.colab import files
uploaded = files.upload()
textfile = list(uploaded.keys())[0]

Saving labeledTrainData.tsv to labeledTrainData.tsv


zip 파일 업로드는 
!unzip labeledTrainData.tsv

25000건 중 5000건 긍정 부정 예측 

In [17]:
import numpy as np
import pandas as pd
df = pd.read_csv(textfile, sep='\t', quoting=3)
df.head(2)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [19]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

## 1. 전처리

In [20]:
#<br /> 테그는 공백으로 변환
df['review'] = df.review.str.replace('<br />', ' ')


In [21]:
#구둣점, 숫자제거 --> 영어 이외의 문자는 공백으로 변환
import re
df['review'] = df.review.apply(lambda x : re.sub('[^A-Za-z]', ' ', x))

In [22]:
df.review[0][:1000]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for 

## 2.학습 및 테스트 데이터 셋 분리

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
df.review, df.sentiment, test_size=0.2,     #2만개는 트레인 5천개는 테스트 25,0000*0.2 = 5천개
 stratify=df.sentiment, #y값을 균등하게 분리하라
random_state=2021
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20000,), (5000,), (20000,), (5000,))

## 3. 텍스트변환

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer



- CountVectorizer, nagram_range=(1,1)

In [27]:
#안되는 방법
cvect1 = CountVectorizer(stop_words='english')
X_train_cv1 = cvect1.fit_transform(X_train)
X_test_cv1 = cvect1.fit_transform(X_test)

In [28]:
X_train_cv1.shape , X_test_cv1.shape #분석결과 값이 달라서 분석이 돌아가지 않음

((20000, 66971), (5000, 37187))

In [30]:
#train , test 데이터 셋에 있는 동일한 단어에 대해서 동일한 값으로 변환하기 위해 되는방법
#반드시  cvect1.fit(X_train)로 먼저 해주고 transform 해줘야함
cvect1 = CountVectorizer(stop_words='english')
cvect1.fit(X_train)
X_train_cv1 = cvect1.transform(X_train)
X_test_cv1 = cvect1.transform(X_test)
X_train_cv1.shape , X_test_cv1.shape 

((20000, 66971), (5000, 66971))

In [40]:
cvect2 = CountVectorizer(stop_words='english', ngram_range=(1,2))
cvect2.fit(X_train)
X_train_cv2 = cvect2.transform(X_train)
X_test_cv2 = cvect2.transform(X_test)
X_train_cv2.shape , X_test_cv2.shape 

((20000, 1461350), (5000, 1461350))

## 4. 로지스틱 회귀모델로 학습(훈련)


In [31]:
from sklearn.linear_model import LogisticRegression


In [37]:
#객체 생성 및 훈련  훈련 데이터의 X와 y모두 필요
lr1 = LogisticRegression(max_iter=500)
%time lr1.fit(X_train_cv1, y_train)

CPU times: user 7.28 s, sys: 8.09 s, total: 15.4 s
Wall time: 7.91 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [38]:
#테스트 데이터에 대해 예측 - 테스트데이터의 X 만 필요
pred1=lr1.predict(X_test_cv1)

- lr2

In [42]:
#객체 생성 및 훈련  훈련 데이터의 X와 y모두 필요
lr2 = LogisticRegression(max_iter=500)
%time lr2.fit(X_train_cv2, y_train)

CPU times: user 1min, sys: 39.5 s, total: 1min 40s
Wall time: 53.1 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [43]:
pred2=lr2.predict(X_test_cv2)

## 5.평가

In [39]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, pred1)
score

0.8832

In [46]:
score2 = accuracy_score(y_test, pred2)
score2

0.8914

## TfidfVectorizer, ngram_range=(1,1)

In [47]:
Tvect1 = TfidfVectorizer(stop_words='english')
Tvect1.fit(X_train)
X_train_Tv1 = Tvect1.transform(X_train)
X_test_Tv1 = Tvect1.transform(X_test)
X_train_Tv1.shape , X_test_Tv1.shape 

((20000, 66971), (5000, 66971))

In [48]:
lr3 = LogisticRegression(max_iter=500)
%time lr3.fit(X_train_Tv1, y_train)

CPU times: user 2.19 s, sys: 2.3 s, total: 4.5 s
Wall time: 2.36 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
pred3=lr3.predict(X_test_Tv1)

In [50]:
score3 = accuracy_score(y_test, pred3)
score3

0.893

TF + Ngram

In [51]:
 

Tvect2 = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
Tvect2.fit(X_train)
X_train_Tv2 = Tvect2.transform(X_train)
X_test_Tv2 = Tvect2.transform(X_test)
X_train_Tv2.shape , X_test_Tv2.shape 

((20000, 1461350), (5000, 1461350))

In [53]:
lr4 = LogisticRegression(max_iter=500)
%time lr4.fit(X_train_Tv2, y_train)

CPU times: user 18.1 s, sys: 11.7 s, total: 29.9 s
Wall time: 16.2 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [54]:
pred4=lr4.predict(X_test_Tv2)

In [55]:
score4 = accuracy_score(y_test, pred4)
score4

0.883

# 6.비지도학습 감성분석

- 감성어휘 사전


In [60]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [61]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sanalyzer = SentimentIntensityAnalyzer()

In [62]:
senti_score= sanalyzer.polarity_scores(df.review[0])
senti_score

{'compound': -0.7943, 'neg': 0.13, 'neu': 0.743, 'pos': 0.127}

In [63]:
df.sentiment[0]

1

In [64]:
#compound score 값이 0.1보다 크면 긍정, 낮으면 부정
def get_sentiment(sanalyzer, review, threshold):
    score = sanalyzer.polarity_scores(review)
    compound_score = score['compound']
    final_socore = 1 if compound_score >= threshold else 0
    return final_socore


In [65]:
df['vader_pred'] =  df.review.apply(lambda x : get_sentiment(sanalyzer, x, 0.1))


In [66]:
df.vader_pred.mean()

0.65588

In [68]:
#평가 , real y - df.sentiment, 예측 y - df.vader_pred
accuracy_score(df.sentiment, df.vader_pred)

0.69476

In [69]:
for threshold in [0, 0.1, 0.2, 0.3]:
    df['vader_pred'] =  df.review.apply(lambda x : get_sentiment(sanalyzer, x, threshold))
    score = accuracy_score(df.sentiment, df.vader_pred)
    print(f'threshold 값 :{threshold} : {score}')
    

threshold 값 :0 : 0.69012
threshold 값 :0.1 : 0.69476
threshold 값 :0.2 : 0.69868
threshold 값 :0.3 : 0.7022
